Final Word2Vec

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore') 

In [3]:
import os 
import numpy as np
import pandas as pd
from sklearn import preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import logging
import pandas as pd
import numpy as np
import gensim
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from bs4 import BeautifulSoup
import csv

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import gmean

In [6]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split()if word not in STOPWORDS) # delete stopwors from text
    return text

In [7]:
from gensim.models import KeyedVectors

In [8]:
pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/GoogleNews-vectors-negative300.bin" 

In [10]:
from gensim.models import Word2Vec, KeyedVectors

In [11]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

vec_king = wv['king']

[=================================================-] 99.9% 1660.5/1662.8MB downloaded


In [13]:
def sent_vectorizer(sent, model):
  sent_vec =[]
  numw = 0
  for w in sent:
    try:
      if numw == 0:
        sent_vec = model[w]
      else:
        sent_vec = np.add(sent_vec, model[w])
      numw+=1
    except:
      pass
  return np.asarray(sent_vec) / numw

In [14]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/data.csv"
df = pd.read_csv(path)
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [15]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [16]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [17]:
#define bins
bins = [0,2,3,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [18]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [19]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [20]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [21]:
df.iloc[:,0] = df.iloc[:,0].apply(clean_text)
print(df.iloc[:100,0])

0     constants alpha beta frac x alpha x+ beta frac...
1     length segment points 2a a4 4 1 2 sqrt 10 unit...
2     chewbacca 20 pieces cherry gum 30 pieces grape...
3     constants b let f x left begin array cl ax + b...
4             calculate sum 1 + 3 + 5 + cdots + 15 + 17
                            ...                        
95    largest value x satisfies equation sqrt 2x 4x ...
96    sum left dfrac 1 3 right + left dfrac 1 3 righ...
97    map 12centimeter length represents 72 kilomete...
98                      find sum values x 2 x23x2 4 x 4
99              given 0 f g 8 f x x2 + 8 g x x2 4 value
Name: Problem , Length: 100, dtype: object


In [22]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

In [23]:
def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split()if word not in STOPWORDS) # delete stopwors from text
    return text

In [24]:
d = [];
for sent in df.iloc[:,0]:
  d.append(sent_vectorizer(sent, wv))

In [25]:
x = pd.DataFrame(d)
x

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.093955,0.054162,-0.004929,0.096340,-0.051677,0.012569,-0.061342,-0.053476,0.025638,0.040121,...,0.057741,0.011684,-0.106701,0.116211,-0.022001,-0.113722,-0.151594,-0.048446,-0.096266,0.144261
1,-0.152155,0.093018,-0.003880,0.125599,-0.081197,-0.001107,-0.040422,-0.061222,-0.025958,0.024775,...,0.056803,-0.022583,-0.120201,0.092927,-0.041892,-0.153596,-0.118248,-0.048664,-0.070796,0.159995
2,-0.170731,0.101694,0.009732,0.150978,-0.051921,0.010126,-0.105255,-0.035787,-0.047395,-0.001214,...,0.073084,-0.053512,-0.120437,0.081855,-0.020905,-0.145200,-0.095649,-0.007357,-0.088108,0.178426
3,-0.174528,0.092311,-0.001647,0.112202,-0.042453,0.029368,-0.086122,-0.026914,-0.009733,0.031288,...,0.056333,0.018444,-0.070353,0.089379,-0.023549,-0.131341,-0.120755,-0.049931,-0.117958,0.147178
4,-0.073251,0.019368,-0.000151,0.112287,-0.064620,-0.008809,-0.102211,-0.118209,0.009707,0.027834,...,0.063856,0.062434,-0.118976,0.090228,-0.043028,-0.109927,-0.136958,-0.050066,-0.085088,0.147312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,-0.092690,0.065202,0.013036,0.143760,-0.048269,0.008990,-0.002005,-0.006887,-0.037829,0.025847,...,0.066554,-0.050393,-0.133339,0.085809,-0.046528,-0.120207,-0.122276,-0.031873,-0.081414,0.151207
1740,0.024542,-0.040936,0.020166,0.105835,-0.040813,0.009846,-0.048888,-0.109009,0.047119,0.072699,...,0.059393,0.032291,-0.142253,0.100879,-0.030029,-0.025464,-0.147693,-0.080807,-0.099524,0.112567
1741,-0.130392,0.050223,0.007564,0.107713,-0.049561,0.018296,-0.053389,-0.040077,-0.021578,0.048746,...,0.055077,0.060960,-0.094589,0.063829,-0.043800,-0.125351,-0.140123,-0.043440,-0.105140,0.121850
1742,-0.110530,0.042288,0.035812,0.127830,-0.074116,0.038634,-0.072896,-0.108733,0.008857,0.013324,...,0.061673,0.010132,-0.107219,0.100161,-0.022954,-0.115289,-0.103913,-0.052344,-0.073305,0.183555


In [26]:
from google.colab import files
x.to_csv('w2v.csv') 
files.download('w2v.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/w2v.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,0,-0.093955,0.054162,-0.004929,0.096340,-0.051677,0.012569,-0.061342,-0.053476,0.025638,...,0.057741,0.011684,-0.106701,0.116211,-0.022001,-0.113722,-0.151594,-0.048446,-0.096266,0.144261
1,1,-0.152155,0.093018,-0.003880,0.125599,-0.081197,-0.001107,-0.040422,-0.061222,-0.025958,...,0.056803,-0.022583,-0.120201,0.092927,-0.041892,-0.153596,-0.118248,-0.048664,-0.070796,0.159995
2,2,-0.170731,0.101694,0.009732,0.150978,-0.051921,0.010126,-0.105255,-0.035787,-0.047395,...,0.073084,-0.053512,-0.120437,0.081855,-0.020905,-0.145200,-0.095649,-0.007357,-0.088108,0.178426
3,3,-0.174528,0.092311,-0.001647,0.112202,-0.042453,0.029368,-0.086122,-0.026914,-0.009733,...,0.056333,0.018444,-0.070353,0.089379,-0.023549,-0.131341,-0.120755,-0.049931,-0.117958,0.147178
4,4,-0.073251,0.019368,-0.000151,0.112287,-0.064620,-0.008809,-0.102211,-0.118209,0.009707,...,0.063856,0.062434,-0.118976,0.090228,-0.043028,-0.109927,-0.136958,-0.050066,-0.085088,0.147312


In [28]:
y=df['Level ']

In [29]:
x.shape

(1744, 301)

In [34]:
x.drop(x.columns[[0]], axis=1, inplace=True)

In [35]:
x.shape

(1744, 300)

**Train Test Split**

In [36]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 300), (524, 300))

In [37]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

3    48.196721
1    29.590164
2    22.213115
Name: Level , dtype: float64


3    46.946565
1    29.961832
2    23.091603
Name: Level , dtype: float64


**Model Building**

In [38]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train) 
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [39]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test)

Perceptron F1 score: 0.598960969354189


**Decision Tree**

In [40]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [41]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [42]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [43]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [44]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [45]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.62541,0.62541,0.648644,0.619527,0.454198,0.454198,0.461057,0.445921


**Random Forest**

In [46]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [47]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [48]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [49]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       271
           3     1.0000    1.0000    1.0000       588

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.6183    0.5159    0.5625       157
           2     0.4545    0.1240    0.1948       121
           3     0.5944    0.8699    0.7063       246

    accuracy                         0.5916       524
   macro avg     0.5558    0.5033    0.4879       524
weighted avg     0.5693    0.5916    0.5451       524



In [50]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.62541,0.62541,0.648644,0.619527,0.454198,0.454198,0.461057,0.445921
1,RandomForest,1.00000,1.00000,1.000000,1.000000,0.591603,0.591603,0.569293,0.545089


**AdaBoost**

In [51]:
clf_adaboost =  AdaBoostClassifier()

In [52]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [53]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [54]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.7287    0.6399    0.6814       361
           2     0.5088    0.3210    0.3937       271
           3     0.6749    0.8401    0.7485       588

    accuracy                         0.6656      1220
   macro avg     0.6374    0.6004    0.6079      1220
weighted avg     0.6539    0.6656    0.6498      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5000    0.4522    0.4749       157
           2     0.2841    0.2066    0.2392       121
           3     0.5612    0.6707    0.6111       246

    accuracy                         0.4981       524
   macro avg     0.4484    0.4432    0.4418       524
weighted avg     0.4789    0.4981    0.4844       524



In [55]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.625410,0.625410,0.648644,0.619527,0.454198,0.454198,0.461057,0.445921
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.591603,0.591603,0.569293,0.545089
2,Adaboost,0.665574,0.665574,0.653902,0.649823,0.498092,0.498092,0.478886,0.484432


**Gradient Boosting Classifier**

In [56]:
clf_gbm = GradientBoostingClassifier()

In [57]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [58]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [59]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9885    0.9529    0.9704       361
           2     0.9910    0.8081    0.8902       271
           3     0.9032    1.0000    0.9492       588

    accuracy                         0.9434      1220
   macro avg     0.9609    0.9203    0.9366      1220
weighted avg     0.9479    0.9434    0.9423      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5605    0.5605    0.5605       157
           2     0.3214    0.1488    0.2034       121
           3     0.6174    0.7805    0.6894       246

    accuracy                         0.5687       524
   macro avg     0.4998    0.4966    0.4844       524
weighted avg     0.5320    0.5687    0.5386       524



In [60]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.625410,0.625410,0.648644,0.619527,0.454198,0.454198,0.461057,0.445921
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.591603,0.591603,0.569293,0.545089
2,Adaboost,0.665574,0.665574,0.653902,0.649823,0.498092,0.498092,0.478886,0.484432
3,GBM,0.943443,0.943443,0.947947,0.942349,0.568702,0.568702,0.531993,0.538558


**SVM Model**

In [61]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [62]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [63]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [64]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5290    0.3795    0.4419       361
           2     0.3161    0.2030    0.2472       271
           3     0.5781    0.7738    0.6618       588

    accuracy                         0.5303      1220
   macro avg     0.4744    0.4521    0.4503      1220
weighted avg     0.5054    0.5303    0.5047      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5841    0.4204    0.4889       157
           2     0.3210    0.2149    0.2574       121
           3     0.5788    0.7764    0.6632       246

    accuracy                         0.5401       524
   macro avg     0.4946    0.4706    0.4698       524
weighted avg     0.5208    0.5401    0.5173       524



In [65]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.625410,0.625410,0.648644,0.619527,0.454198,0.454198,0.461057,0.445921
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.591603,0.591603,0.569293,0.545089
2,Adaboost,0.665574,0.665574,0.653902,0.649823,0.498092,0.498092,0.478886,0.484432
3,GBM,0.943443,0.943443,0.947947,0.942349,0.568702,0.568702,0.531993,0.538558
4,SVM,0.530328,0.530328,0.505380,0.504653,0.540076,0.540076,0.520841,0.517271


**KNN**

In [66]:
knn = KNeighborsClassifier(n_neighbors=7)

In [67]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [68]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [69]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6386    0.5873    0.6118       361
           2     0.5633    0.3284    0.4149       271
           3     0.6822    0.8469    0.7557       588

    accuracy                         0.6549      1220
   macro avg     0.6280    0.5875    0.5941      1220
weighted avg     0.6429    0.6549    0.6374      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5606    0.4713    0.5121       157
           2     0.3636    0.1983    0.2567       121
           3     0.5798    0.7683    0.6608       246

    accuracy                         0.5477       524
   macro avg     0.5013    0.4793    0.4765       524
weighted avg     0.5241    0.5477    0.5230       524



In [70]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.625410,0.625410,0.648644,0.619527,0.454198,0.454198,0.461057,0.445921
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.591603,0.591603,0.569293,0.545089
2,Adaboost,0.665574,0.665574,0.653902,0.649823,0.498092,0.498092,0.478886,0.484432
3,GBM,0.943443,0.943443,0.947947,0.942349,0.568702,0.568702,0.531993,0.538558
4,SVM,0.530328,0.530328,0.505380,0.504653,0.540076,0.540076,0.520841,0.517271
5,KNN,0.654918,0.654918,0.642868,0.637427,0.547710,0.547710,0.524112,0.522952


**GNB**

In [71]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [72]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [73]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [74]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4729    0.3629    0.4107       361
           2     0.2973    0.1218    0.1728       271
           3     0.5529    0.7823    0.6479       588

    accuracy                         0.5115      1220
   macro avg     0.4410    0.4223    0.4104      1220
weighted avg     0.4725    0.5115    0.4722      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5194    0.4268    0.4685       157
           2     0.3333    0.1322    0.1893       121
           3     0.5620    0.7927    0.6577       246

    accuracy                         0.5305       524
   macro avg     0.4716    0.4506    0.4385       524
weighted avg     0.4964    0.5305    0.4929       524



In [75]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.625410,0.625410,0.648644,0.619527,0.454198,0.454198,0.461057,0.445921
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.591603,0.591603,0.569293,0.545089
2,Adaboost,0.665574,0.665574,0.653902,0.649823,0.498092,0.498092,0.478886,0.484432
3,GBM,0.943443,0.943443,0.947947,0.942349,0.568702,0.568702,0.531993,0.538558
4,SVM,0.530328,0.530328,0.505380,0.504653,0.540076,0.540076,0.520841,0.517271
5,KNN,0.654918,0.654918,0.642868,0.637427,0.547710,0.547710,0.524112,0.522952
6,GNB,0.511475,0.511475,0.472450,0.472154,0.530534,0.530534,0.496408,0.492859
